In [13]:
# Imports here
import matplotlib.pyplot as plt
import numpy as np
import torch
from torch import nn
from torch import optim
import torch.nn.functional as F
from torchvision import datasets, transforms, models
import torchvision
import torchvision.models as models
import torchvision.transforms as transforms 

from PIL import Image
import json
from matplotlib.ticker import FormatStrFormatter



In [14]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cuda:0


# **Steps**

Step 1: Load Dataset <p>
Step 2: Transform the Dataset <p>
Step 3: Create Model <p>
Step 4: Train Model <p>
Step 5: Save the Model <p>
Step 6: Load the Model <p>


# Step 1: Load Dataset



In [15]:
data_dir = './dataset'
train_dir = data_dir + '/train'
valid_dir = data_dir + '/valid'
test_dir = data_dir + '/test'

In [16]:
with open('cat_to_name.json', 'r') as f:
    cat_to_name = json.load(f)

# Step 2: Transform the Dataset

The pre-trained networks you'll use were trained on the ImageNet dataset where each color channel was normalized separately. For all three sets you'll need to normalize the means and standard deviations of the images to what the network expects. For the means, it's [0.485, 0.456, 0.406] and for the standard deviations [0.229, 0.224, 0.225], calculated from the ImageNet images. These values will shift each color channel to be centered at 0 and range from -1 to 1.


In [17]:
# Define your transforms for the training, validation, and testing sets
train_transforms = transforms.Compose([transforms.RandomRotation(30),
                                       transforms.RandomResizedCrop(224),
                                       transforms.RandomHorizontalFlip(),
                                       transforms.ToTensor(),
                                       transforms.Normalize([0.485, 0.456, 0.406], 
                                                            [0.229, 0.224, 0.225])])

test_transforms = transforms.Compose([transforms.Resize(256),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize([0.485, 0.456, 0.406], 
                                                           [0.229, 0.224, 0.225])])

validation_transforms = transforms.Compose([transforms.Resize(256),
                                            transforms.CenterCrop(224),
                                            transforms.ToTensor(),
                                            transforms.Normalize([0.485, 0.456, 0.406], 
                                                                 [0.229, 0.224, 0.225])])


trainset = torchvision.datasets.ImageFolder(root=train_dir, transform=train_transforms)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True, num_workers=0)

testset = torchvision.datasets.ImageFolder(root=valid_dir, transform=test_transforms)
validloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=False, num_workers=0)

# Step 3: Create Model



In [18]:
class CNNModel(nn.Module):
    def __init__(self):
        super(CNNModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, kernel_size=3, padding=1)
        self.conv2 = nn.Conv2d(32, 64, kernel_size=3, padding=1)
        self.conv3 = nn.Conv2d(64, 128, kernel_size=3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(128 * 56 * 56, 512)  # Adjusted this line
        self.fc2 = nn.Linear(512, 102)
        self.dropout = nn.Dropout(0.2)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.pool(self.relu(self.conv2(x)))
        x = self.pool(self.relu(self.conv3(x)))
        x = x.view(x.size(0), -1)
        x = self.relu(self.fc1(x))
        x = self.dropout(x)
        x = self.fc2(x)
        return x





In [19]:
model = CNNModel().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [21]:
# Model instantiation and dummy input
model_test = CNNModel()
input_test = torch.randn(64, 3, 224, 224)  # Random input tensor

# Forward pass
output_test = model_test(input_test)
print(output_test.shape)


torch.Size([64, 102])


# Step 4: Train Model

In [22]:
import torch
def mixup_data(x, y, alpha=1.0):
    if alpha > 0:
        lam = np.random.beta(alpha, alpha)
    else:
        lam = 1

    batch_size = x.size()[0]
    index = torch.randperm(batch_size)

    mixed_x = lam * x + (1 - lam) * x[index, :]
    y_a, y_b = y, y[index]
    return mixed_x, y_a, y_b, lam

num_epochs = 1

# Set model to train mode
model.train()

for epoch in range(num_epochs):
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # Unpack data
        inputs, labels = data[0].to(device),data[1].to(device)
        #print(inputs.shape)
        #print(labels.shape)
        # Apply mixup
        inputs_mixed, labels_a, labels_b, lam = mixup_data(inputs, labels, alpha=1.0)
        

        # Make sure there's no size mismatch
        assert inputs_mixed.size(0) == labels_a.size(0) == labels_b.size(0), "Size mismatch after mixup!"
        
        # Clear gradients
        optimizer.zero_grad()

        # Forward pass
        outputs = model(inputs_mixed)

        # Compute mixup loss
        loss = lam * criterion(outputs, labels_a) + (1 - lam) * criterion(outputs, labels_b)
        
        # Backward pass
        loss.backward()
        
        # Update weights
        optimizer.step()
        
        running_loss += loss.item()
        
        # Print loss every 100 batches
        if (i+1) % 100 == 0:
            print('[%d, %5d] loss: %.3f' % (epoch + 1, i + 1, running_loss / 100))
            running_loss = 0.0
        # print("Inputs Mixed Shape:", inputs_mixed.shape)
        # print("Labels A Shape:", labels_a.shape)
        # print("Labels B Shape:", labels_b.shape)
        # print("Model Outputs Shape:", outputs.shape)

print('Finished Training')


In [ ]:
dataiter = iter(trainloader)
images, labels = next(dataiter)
print(images.shape)
print(labels.shape)

torch.Size([64, 3, 224, 224])
torch.Size([64])


In [ ]:
correct = 0
total = 0

with torch.no_grad():
    for data in validloader:
        inputs, labels = data[0].to(device), data[1].to(device)

        outputs = model(inputs)
        # Remove the reshaping line if it's not needed
        # outputs = outputs.view(inputs.shape[0], -1)

        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print('Accuracy of the network on the %d test images: %d %%' % (total, 100 * correct / total))


Accuracy of the network on the 818 test images: 5 %


# Step 5: Save the Model

Now that your network is trained, save the model so you can load it later for making predictions. You probably want to save other things such as the mapping of classes to indices which you get from one of the image datasets: image_datasets['train'].class_to_idx. You can attach this to the model as an attribute which makes inference easier later on.

model.class_to_idx = image_datasets['train'].class_to_idx

Remember that you'll want to completely rebuild the model later so you can use it for inference. Make sure to include any information you need in the checkpoint. If you want to load the model and keep training, you'll want to save the number of epochs as well as the optimizer state, optimizer.state_dict. You'll likely want to use this trained model in the next part of the project, so best to save it now.

In [ ]:


torch.save(model,'./mixup.pth')

# Step 6: Load the Model
At this point it's good to write a function that can load a checkpoint and rebuild the model. That way you can come back to this project and keep working on it without having to retrain the network.

In [ ]:
model = torch.load('epoch100.pth').to(device)
# If you also saved other attributes like optimizer state
model.state_dict()



OrderedDict([('conv1.weight',
              tensor([[[[ 2.7469e-01,  7.6502e-03,  1.3433e-01],
                        [ 7.2101e-02, -1.8143e-01,  9.3182e-02],
                        [ 1.5716e-02, -2.0233e-02, -2.1285e-01]],
              
                       [[ 2.4770e-01, -7.9254e-02, -1.7063e-01],
                        [ 2.4105e-01,  2.6078e-03, -2.2444e-01],
                        [-6.3570e-02,  1.0545e-01, -1.6039e-01]],
              
                       [[ 1.0507e-01,  1.2329e-01,  1.8267e-01],
                        [ 6.1663e-02, -1.8857e-01, -1.7566e-01],
                        [ 6.2167e-02, -5.3986e-02, -2.0984e-01]]],
              
              
                      [[[-1.8172e-02, -9.9116e-02,  2.9565e-02],
                        [-1.1903e-01,  1.2689e-01,  7.7647e-02],
                        [ 1.0105e-01,  2.0074e-01,  1.1598e-01]],
              
                       [[ 1.7324e-01,  2.0787e-01, -1.7741e-01],
                        [ 1.1066e-01,  1.8599